In [1]:
# Установка зависимостей
%pip install torch torchvision pingouin scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 4.8 MB/s eta 0:00:00a 0:00:01


In [3]:
!pip install git+https://github.com/VanessB/pytorch-fd.git

  Cloning https://github.com/VanessB/pytorch-fd.git to /tmp/pip-req-build-fpowush5
  Running command git clone --filter=blob:none --quiet https://github.com/VanessB/pytorch-fd.git /tmp/pip-req-build-fpowush5
  Resolved https://github.com/VanessB/pytorch-fd.git to commit 477ec5229231acfa632d08dbb953c30716b182bd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torchfd: filename=torchfd-2025.8.1-py3-none-any.whl size=33144 sha256=be2b065704202347239817dd97b95a73c8eec3a6401600a69a2d61e8e400e0d1
  Stored in directory: /tmp/pip-ephem-wheel-cache-8xuh7pju/wheels/3d/d1/cb/a4c920016f2d6d02bd5888ac420ea64767dd8e5da4d1ae689b
Successfully built torchfd


In [4]:
import torchfd

In [5]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pandas as pd

from architectures import get_architecture_factory
from modules import Conv2dEmbedder, DenseClassifier
from training import train_infomax_embedder, convert_to_embeddings, classification_callback
from plots import plot_history, plot_embeddings
from statistics import MultivariateNormalTests
import infomax.channels

ModuleNotFoundError: No module named 'architectures'

In [7]:
import torchfd

ModuleNotFoundError: No module named 'torchfd'

In [ ]:
# Проверка GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Устройство: {device}")

Устройство: cpu


In [5]:
# Загрузка CIFAR-10
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch.utils.data import DataLoader

# Аугментации и преобразования
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Датасеты
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

# DataLoader'ы
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

print(f"Размер тренировочного датасета: {len(train_dataset)}")
print(f"Размер тестового датасета: {len(test_dataset)}")

  0%|          | 426k/170M [00:00<04:52, 582kB/s] 


KeyboardInterrupt: 

In [ ]:
# БАЗОВАЯ КОНФИГУРАЦИЯ (одинаковая для всех экспериментов)
base_config = {
    "embedding_dim": 128,
    "batch_size_train": 512,
    "n_epochs": 100,  # можно уменьшить для теста
    "input_p": 0.2,
    "output_p": 0.1,
    "marginalizer": "OuterProductMarginalizer",
    "distribution": "normal",
    "discriminator_network_inner_dim": 256,
    "discriminator_network_output_dim": 64
}

# АРХИТЕКТУРЫ ДЛЯ СРАВНЕНИЯ
architectures_to_test = [
    "DotProduct",           # ultra-simple
    "Bilinear",             # simple  
    "AdditiveGaussainT",    # theoretical
    "SeparableT",           # medium
    "DenseT",               # complex
    "VeryDenseT"            # ultra-complex
]

print("Будем тестировать архитектуры:", architectures_to_test)

In [ ]:
def run_single_experiment(architecture_name, config, train_loader, test_loader, device):
    """
    Запускает один полный эксперимент с заданной архитектурой
    """
    print(f"\n{'='*50}")
    print(f"🚀 ЗАПУСКАЕМ ЭКСПЕРИМЕНТ: {architecture_name}")
    print(f"{'='*50}")
    
    # 1. СОЗДАЁМ МОДЕЛЬ
    backbone = Conv2dEmbedder(embedding_dim=config["embedding_dim"])
    normalization = torch.nn.BatchNorm1d(config["embedding_dim"], affine=False)
    embedder_network = torch.nn.Sequential(backbone, normalization)
    
    # 2. ПОЛУЧАЕМ КЛАССИФИКАТОР СРАВНЕНИЯ
    factory = get_architecture_factory(config["embedding_dim"], config)
    discriminator_network = factory[architecture_name]()
    
    # 3. СОБИРАЕМ ПОЛНУЮ МОДЕЛЬ
    model = infomax.embeddings.Embedder(
        embedder_network,
        discriminator_network,
        infomax.channels.BoundedVarianceGaussianChannel(config["input_p"]),
        infomax.channels.BoundedVarianceGaussianChannel(config["output_p"]),
    ).to(device)
    
    # 4. ЗАПУСКАЕМ ОБУЧЕНИЕ
    history = train_infomax_embedder(
        model,
        train_loader,
        test_loader,
        device,
        n_epochs=config["n_epochs"],
        callback=classification_callback,
        distribution=config["distribution"]
    )
    
    # 5. СОХРАНЯЕМ РЕЗУЛЬТАТЫ
    results = {
        'architecture': architecture_name,
        'model': model,
        'history': history,
        'final_embeddings': convert_to_embeddings(model.embedder_network, test_loader, device)
    }
    
    print(f"✅ {architecture_name} завершён!")
    return results

In [ ]:
# ЗАПУСКАЕМ ВСЕ АРХИТЕКТУРЫ
all_results = {}

for arch_name in architectures_to_test:
    try:
        results = run_single_experiment(arch_name, base_config, train_loader, test_loader, device)
        all_results[arch_name] = results
    except Exception as e:
        print(f"❌ Ошибка с {arch_name}: {e}")
        continue

print(f"\n🎯 Успешно завершено экспериментов: {len(all_results)}/{len(architectures_to_test)}")

In [ ]:
# СОЗДАЁМ СВОДНУЮ ТАБЛИЦУ РЕЗУЛЬТАТОВ
summary_data = []

for arch_name, results in all_results.items():
    history = results['history']
    
    # Извлекаем финальные метрики
    try:
        # Accuracy (берём последнее значение)
        if 'classification_logistic_regression' in history and 'accuracy' in history['classification_logistic_regression']:
            accuracy_data = history['classification_logistic_regression']['accuracy']
            final_accuracy = accuracy_data[-1][2] if accuracy_data else 0.0
        else:
            final_accuracy = 0.0
        
        # Normalcy test (берём последнее значение)
        if 'distribution' in history and 'henze_zirkler_(test)' in history['distribution']:
            normalcy_data = history['distribution']['henze_zirkler_(test)']
            final_normalcy = normalcy_data[-1][2] if normalcy_data else 0.0
        else:
            final_normalcy = 0.0
        
        # Loss (берём последнее значение)
        if 'training' in history and 'loss' in history['training']:
            loss_data = history['training']['loss']
            final_loss = loss_data[-1][2] if loss_data else 0.0
        else:
            final_loss = 0.0
        
        summary_data.append({
            'Architecture': arch_name,
            'Accuracy': final_accuracy,
            'Normalcy_p-value': final_normalcy,
            'Final_Loss': final_loss
        })
        
    except Exception as e:
        print(f"Ошибка при анализе {arch_name}: {e}")
        continue

# СОЗДАЁМ И ВЫВОДИМ ТАБЛИЦУ
summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values('Accuracy', ascending=False)

print("📊 СВОДНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ:")
print("="*60)
print(summary_df.round(4))

In [ ]:
# ГРАФИКИ ДЛЯ АНАЛИЗА ГИПОТЕЗЫ
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Accuracy vs Architecture
axes[0,0].bar(summary_df['Architecture'], summary_df['Accuracy'], color='skyblue', alpha=0.7)
axes[0,0].set_title('Accuracy по архитектурам')
axes[0,0].set_ylabel('Accuracy')
axes[0,0].tick_params(axis='x', rotation=45)

# 2. Normalcy vs Architecture
axes[0,1].bar(summary_df['Architecture'], summary_df['Normalcy_p-value'], color='lightcoral', alpha=0.7)
axes[0,1].set_title('Нормальность распределений по архитектурам')
axes[0,1].set_ylabel('p-value нормальности')
axes[0,1].tick_params(axis='x', rotation=45)

# 3. КОРРЕЛЯЦИЯ: Accuracy vs Normalcy
axes[1,0].scatter(summary_df['Normalcy_p-value'], summary_df['Accuracy'], s=100, alpha=0.7)
for i, row in summary_df.iterrows():
    axes[1,0].annotate(row['Architecture'], 
                      (row['Normalcy_p-value'], row['Accuracy']),
                      xytext=(5, 5), textcoords='offset points')
axes[1,0].set_xlabel('p-value нормальности')
axes[1,0].set_ylabel('Accuracy')
axes[1,0].set_title('Корреляция: Нормальность ↔ Accuracy')
axes[1,0].grid(True, alpha=0.3)

# 4. Loss по архитектурам
axes[1,1].bar(summary_df['Architecture'], summary_df['Final_Loss'], color='lightgreen', alpha=0.7)
axes[1,1].set_title('Final Loss по архитектурам')
axes[1,1].set_ylabel('Loss')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# СРАВНЕНИЕ ПРОСТРАНСТВА ЭМБЕДДИНГОВ ДЛЯ ЛУЧШЕЙ И ХУДШЕЙ АРХИТЕКТУРЫ
best_arch = summary_df.iloc[0]['Architecture']
worst_arch = summary_df.iloc[-1]['Architecture']

print(f"🎯 Лучшая архитектура: {best_arch}")
print(f"📉 Худшая архитектура: {worst_arch}")

fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Эмбеддинги лучшей архитектуры
best_embeddings, best_labels = all_results[best_arch]['final_embeddings']
best_embeddings = best_embeddings.detach().cpu().numpy()
best_labels = best_labels.detach().cpu().numpy()

# Эмбеддинги худшей архитектуры  
worst_embeddings, worst_labels = all_results[worst_arch]['final_embeddings']
worst_embeddings = worst_embeddings.detach().cpu().numpy()
worst_labels = worst_labels.detach().cpu().numpy()

# Визуализация
classes = np.unique(best_labels)
for label in classes:
    # Лучшая архитектура
    selected = best_embeddings[best_labels==label]
    axes[0].scatter(selected[:,0], selected[:,1], label=f'Class {label}', alpha=0.6, s=30)
    
    # Худшая архитектура
    selected = worst_embeddings[worst_labels==label] 
    axes[1].scatter(selected[:,0], selected[:,1], label=f'Class {label}', alpha=0.6, s=30)

axes[0].set_title(f'Эмбеддинги: {best_arch} (Accuracy: {summary_df.iloc[0]["Accuracy"]:.3f})')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].set_title(f'Эмбеддинги: {worst_arch} (Accuracy: {summary_df.iloc[-1]["Accuracy"]:.3f})')  
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.show()

In [ ]:
# СТАТИСТИЧЕСКИЙ АНАЛИЗ РЕЗУЛЬТАТОВ
print("📈 СТАТИСТИЧЕСКИЙ АНАЛИЗ:")
print("="*50)

# Корреляция между нормальностью и accuracy
correlation = np.corrcoef(summary_df['Normalcy_p-value'], summary_df['Accuracy'])[0,1]
print(f"📊 Корреляция между нормальностью и accuracy: {correlation:.3f}")

# Находим оптимальную архитектуру
optimal_idx = summary_df['Accuracy'].idxmax()
optimal_arch = summary_df.loc[optimal_idx, 'Architecture']
optimal_accuracy = summary_df.loc[optimal_idx, 'Accuracy']
optimal_normalcy = summary_df.loc[optimal_idx, 'Normalcy_p-value']

print(f"\n🎯 ОПТИМАЛЬНАЯ АРХИТЕКТУРА: {optimal_arch}")
print(f"   • Accuracy: {optimal_accuracy:.3f}")
print(f"   • p-value нормальности: {optimal_normalcy:.3f}")

# Анализ спектра сложности
simple_archs = ['DotProduct', 'Bilinear', 'AdditiveGaussainT']
complex_archs = ['DenseT', 'VeryDenseT']
medium_archs = ['SeparableT']

simple_avg = summary_df[summary_df['Architecture'].isin(simple_archs)]['Accuracy'].mean()
medium_avg = summary_df[summary_df['Architecture'].isin(medium_archs)]['Accuracy'].mean()  
complex_avg = summary_df[summary_df['Architecture'].isin(complex_archs)]['Accuracy'].mean()

print(f"\n📈 СРЕДНЯЯ ACCURACY ПО ГРУППАМ СЛОЖНОСТИ:")
print(f"   • Простые: {simple_avg:.3f}")
print(f"   • Средние: {medium_avg:.3f}")
print(f"   • Сложные: {complex_avg:.3f}")

# ВЫВОДЫ
print(f"\n🎪 ВЫВОДЫ:")
print("="*30)
if correlation > 0.5:
    print("✅ Гипотеза ПОДТВЕРЖДЕНА: Высокая корреляция между нормальностью и accuracy")
else:
    print("❌ Гипотеза НЕ подтверждена: Низкая корреляция между нормальностью и accuracy")

if medium_avg > simple_avg and medium_avg > complex_avg:
    print("✅ Найдена 'золотая середина' сложности!")
elif simple_avg > complex_avg:
    print("⚠️  Простые архитектуры работают лучше сложных")
else:
    print("⚠️  Сложные архитектуры работают лучше простых")

In [ ]:
# СОХРАНЕНИЕ РЕЗУЛЬТАТОВ ДЛЯ ОТЧЁТА
import json
from datetime import datetime

# Создаём папку для результатов
results_dir = f"results_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
import os
os.makedirs(results_dir, exist_ok=True)

# Сохраняем сводную таблицу
summary_df.to_csv(f'{results_dir}/summary.csv', index=False)

# Сохраняем конфигурацию
with open(f'{results_dir}/config.json', 'w') as f:
    json.dump(base_config, f, indent=2)

# Сохраняем графики
plt.figure(figsize=(10, 6))
plt.bar(summary_df['Architecture'], summary_df['Accuracy'])
plt.title('Accuracy по архитектурам')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(f'{results_dir}/accuracy_comparison.png', dpi=300, bbox_inches='tight')

print(f"💾 Результаты сохранены в папку: {results_dir}")
print("📁 Содержимое:")
print("   - summary.csv - сводная таблица")
print("   - config.json - конфигурация эксперимента") 
print("   - accuracy_comparison.png - график сравнения")